# Preparing Libraries and Installations

In [1]:
%pip install bs4 selenium

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import pandas as pd
import time

# Opening the Website

In [8]:
# URL Ulasan dari Female Daily
url = "https://reviews.femaledaily.com/products/moisturizer/sun-protection-1/emina/sun-protection-spf-30-pa"

try:
    # Inisialisasi option chrome
    chrome_options = Options()
    # chrome_options.add_argument('--headless')
    # chrome_options.add_argument('--no-sandbox')
    # chrome_options.add_argument('--disable-dev-shm-usage')

    # Inisialisasi WebDriver
    service = Service(executable_path = ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Buka URL
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    # Ekstrak dan print title
    print(driver.title)
    time.sleep(5) #tunggu 5 detik

    # Get List Reviews
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list_reviews = soup.find(class_='list-reviews')
    reviews = list_reviews.find_all(class_='review-card')

    for review in reviews:
        right_col = review.find(class_= 'review-card-col right')
        wrapper = right_col.find(class_='review-content-wrapper')

        review_text = wrapper.find(class_='text-content').get_text(strip=True)
        print(f'Review: {review_text}\n')

    # list_reviews = driver.find_elements(By.CLASS_NAME, 'list-reviews')
    # reviews = list_reviews.find_elements(By.CLASS_NAME, 'review-card')

    # for review in reviews:
    #     age = review.find_element(By.CLASS_NAME, 'profile_age')
    #     print(f"Age: {age}\n")

    # Next Button untuk pagination
    # next_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'paging-prev-text-active')))

    # driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
    # time.sleep(1)  # brief sleep to allow any potential animations to complete

    # try:
    #     next_button.click()
    #     time.sleep(10)
    # except WebDriverException:
    #     # If the click fails, use JavaScript to click
    #     driver.execute_script("arguments[0].click();", next_button)

#exception
except WebDriverException as e:
    print(f"WebDriverException: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
except TimeoutException:
    print("The element was not clickable within the given time")
finally:
    time.sleep(10)
    driver.quit()

Emina Sun Protection SPF 30 PA+++ - Beauty Review
Review: Teksturnya agak watery gituDiblendnya agak lumayan usahaDi muka saya, kerasa panas pas dipakai. Dan ada aroma agak tajemnya. Tapi emang harganya sangat affordable. Kalau pas keringetan, lumayan keliatan putih putih gitu bulir keringetnya dan agak pedih dimata.

Review: udah pake ini dari jaman SMA hahah seneng banget bisa cocok. suka banget sama texturenya yang ringan, gampang nyerep, gampang diblend, gak bikin gerah, gak ada whitecast. yang bikin seneng lagi, harganya murah meriah dan gampang didapetin.

Review: ini adalah suncreen pertamaku , dulu cocok banget krn harganya sangat friendly di kantong pelajar , mudah untuk di apply juga .. cuma gatau kenapa tiap aku habis pakai ini muka jadi keliatan lebih kusam , jd aku ga terlalu merekomendasikannya siii

Review: Emina itu brand yang nge hype buat remaja dan aku dulu pernah jadi one of among a lot collector juga diantaranya yg pernah aku beli ini adalah spf nya yang di edisi p

# Trying Selenium